In [ ]:
import os
import json
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from enum import Enum

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

In [ ]:
class Model(Enum):
    GPT_4o_MINI = 'gpt-4o-mini'
    LLAMA_3_2 = 'llama3.2'

class ModelClient:
    def __init__(self, model: str, **kwargs):
        self.client = OpenAI(**kwargs)
        self.model = model

    def chat(self, messages, **kwargs):
        return self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            **kwargs
        )

In [ ]:
def get_open_ai_client(model: Model) -> ModelClient:
    match model:
        case Model.GPT_4o_MINI:
            return ModelClient(model=Model.GPT_4o_MINI.value)
        case Model.LLAMA_3_2:
            return ModelClient(model=Model.LLAMA_3_2.value, base_url='http://localhost:11434/v1', api_key='ollama')
        case _:
            raise ValueError(f"Unsupported model: {model}")

In [ ]:
system_prompt = "You are a tutor expert at providing answers to technical question in a detailed manner. \
Your goal is to provide answers that helps your syudents build understanding of any question you are asked \
by providing beginner friendly explanations with relevant examples. In addition, you are able to suggests \
further topics or references to solidify their understanding. The question could be code snippet that the \
student is trying to understand what it does. Ensure your answers are broken down in a markdown manner"

In [ ]:
def get_user_prompt():
    question = input('Enter your question: ')
    return f"Please provide answer to this question to the best of your knowledge: {question}."


In [ ]:
def answer_question(model: Model, user_prompt, stream=False):
    model_client = get_open_ai_client(model);

    response = model_client.chat(
        messages=[
            {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ],
    stream=stream
    )   

    if stream:
        stream_answer(response)
    else:
        answer = response.choices[0].message.content
        display(Markdown(answer))


def stream_answer(stream_response):
    answer = ""
    display_handle = display(Markdown(""), display_id=True)

    for chunk in stream_response:
        answer += chunk.choices[0].delta.content or ''
        answer = answer.replace("```","")
        update_display(Markdown(answer), display_id=display_handle.display_id)

In [ ]:
user_question = get_user_prompt()

display(Markdown("## **Response from GPT 4o Mini**")) 
gpt_4o_mini_response = answer_question(model=Model.GPT_4o_MINI, user_prompt=user_question, stream=True)
display(Markdown("## **Response from GPT 4o Mini ends here**")) 

display(Markdown("## **Response from LLAMA 3.2**")) 
llama_response = answer_question(model=Model.LLAMA_3_2, user_prompt=user_question, stream=True)
display(Markdown("## **Response from LLAMA 3.2 ends here**")) 



